In [67]:
from utils import *
import random
import csv
import time
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

In [68]:
def compute_query_score(query,utility_matrix,queries,usersID,k):
    queriesIDs=queries['queryID'].values.tolist()
    queries=queries['query'].values.tolist()
    #get a set of queries
    query_list=[]
    for i in queries:
        query={}
        for x in i.split(','):
            aux=x.split('=')
            query.update({aux[0]:aux[1]})
        query_list.append(query)
    #compute query similarity 
    values_similarity=[]
    for i in query_list:
        values_similarity.append(jaccard_similarity(query.values(),i.values()))
    values_similarity=np.array(values_similarity)
    queriesIDs=np.array(queriesIDs)
    #test if the query is already scored
    if values_similarity.max()==1:
        return utility_matrix[queriesIDs[values_similarity.argmax()]]
    else:
        #get the k nearest neighbor
        values_similarity,queriesIDs=KNN(k,values_similarity,queriesIDs)
        #compute scores
        scores=(np.array(utility_matrix[queriesIDs])@values_similarity)/values_similarity.sum()
    return pd.DataFrame(scores, index=usersID,columns=['Q2001'])

In [69]:
um= pd.read_csv('csv\\predicted_utility_matrix.csv',index_col=0)
users = pd.read_csv("./csv/users.csv",index_col=False)
usersID=users['User_Id'].values.tolist()
dataset=pd.read_csv("./csv/dataset.csv",index_col=False)


In [70]:
def transfrom_query(query):
    output={}
    for x in query.split(','):
            aux=x.split('=')
            output.update({aux[0]:aux[1]})
    return output

In [77]:
queries=pd.read_csv("./csv/queries.csv",index_col=False)
queriesIDs=um.columns
rmse=0
for i,Id in enumerate(queriesIDs):
    query=transfrom_query(queries['query'].iloc[i])
    df=compute_query_score(query,um.loc[:, ~um.columns.isin([Id])],queries,usersID,400)
    rmse+=sqrt(mean_squared_error(np.array(df),np.array(um[Id])))
rmse=rmse/2000

KeyError: 'Q2000'

In [73]:
new_query={'name':'Isa','city':'Trento','job':'Producer'}

In [76]:
compute_query_score(new_query,um,queries,usersID,400)

U1       67.00
U2       88.23
U3        1.00
U4       78.00
U5       21.00
         ...  
U1996    86.95
U1997    86.91
U1998    87.49
U1999    13.16
U2000    63.21
Name: Q2000, Length: 2000, dtype: float64